In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

import time

kafka_topic_name = "Topic_kita"
kafka_bootstrap_servers = 'localhost:9092'

spark = SparkSession \
    .builder \
    .appName("Structured Streaming") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.3.1") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Construct a streaming DataFrame that reads from the topic
flower_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic_name) \
    .option("startingOffsets", "latest") \
    .load()

flower_df1 = flower_df.selectExpr("CAST(value AS STRING)", "timestamp")

flower_schema_string = "order_id INT, sepal_length DOUBLE, sepal_width DOUBLE, petal_length DOUBLE, petal_width DOUBLE, species STRING"

flower_df2 = flower_df1 \
    .select(from_csv(col("value"), flower_schema_string) \
    .alias("flower"), "timestamp")

flower_df3 = flower_df2.select("flower.*", "timestamp")

flower_df3.createOrReplaceTempView("flower_find")

# Menggunakan DataFrame flower_df3
song_find_text = flower_df3

# Menambahkan operasi agregasi (misalnya, hitung jumlah data)
flower_agg_df = song_find_text.groupBy("species").count()

flower_agg_write_stream = flower_agg_df \
    .writeStream \
    .trigger(processingTime='5 seconds') \
    .outputMode("complete") \
    .option("truncate", "false") \
    .format("memory") \
    .queryName("testedTableAgg") \
    .start()

flower_agg_write_stream.awaitTermination(1)

In [ ]:
df = spark.sql("SELECT * FROM testedTableAgg")  # Menggunakan nama query yang baru
df.show()

In [ ]:
df = spark.sql("SELECT * FROM testedTable")
df.show(3)

In [ ]:
df_count = df.count()
df_count.show()

In [ ]:
df_count = df.count()
df_count.show()